# Building Khmer Word Segmentation using Conditional Random Fields
This notebook contains the complete code that can be run to see the results as part of the approach to make this a reproducable research. Instruction is included on how to setup and run in Google Colab or any other python notebook compatible environments.

This notebook conains two sections. 
1. **Report**:  covers the report potion of this study which describe the approach, the algorithm and the results.
1. **Code**: shows the code which detail  the setup so you can follow along and see the excution results. You can execute the code in this Python notebook directly from your browser when setup to use Google Colab or other compatible systems. See implementation notebook "CRF-Khmer-Segmentation.ipynb".

Expand each section and subsection to see detail.



# Report

## Abstract

The motivation for this study is to look at different implementation approaches of Conditional Random Fields (CRF) to segment Khmer document which showed in earlier paper by Vichet Chea et al. (2015) to have a very high accuracy.

We first describe the approach to gather training data from Khmer news site and save them to Github to make the corpus available for future study. The data is parsed using existing tool avaiable by the author above. This save us times from manually segmented the text.

Then we explore the different features to be used in CRF and its performances. We also compare two different approaches of using a character based and Khmer character cluster (KCC) based. This implimentation with KCC approach shows an accuracy of 99.7% F-score comparable to performance showed 98.5% in earlier paper.


## Introduction

Khmer language (Cambodian) is written without spaces between words like Thai or Chinese writing system. Khmer may have spaces between phrases to indicate pauses but not specificaly between words. For native speaker, it is easy to discern words from the long sequences of characters. It is a challege to teach computer to recognize words with standard program technique since it is difficult to program specific logic to describe how to separate words.

Since they are from different sources/sites, the format in term of how the text is formatted with invisible spaces are not consistent. 

We created this series of of blog post as an introduction on different approaches to word sgementation: https://medium.com/@phylypo/segmentation-of-khmer-text-using-conditional-random-fields-3a2d4d73956a.

The Conditional Random Field (CRF) has several advantages compare to ngrams, MEMM for sequence related problems. We use this approach to find the most probable sequences of labels for a given sequences of characters seen in the training data.

### Overview
The general idea is to use CRF to learn how to segment Khmer text based on our training data.
First we get a list of Khmer articles, then created a segmented version which contain spaces between each word. Then create a format that the algorithm can learn from. 

For example, the sentence: ```This is a test.```
becomes:
```
T h i s i s a t e s t .
1 0 0 0 1 0 1 1 0 0 0 1
```
The label 1 indicates  that the corresponding character is the beginning of the word. The goal is for the algorithm to learn the feature and able to predict "0" or "1" on other sentences. 
For example, if we passed in: ```Whatisthis?```
The correct prediction would be:
```
W h a t i s t h i s ?
1 0 0 0 1 0 1 0 0 0 1
```

## Tools

We are not going to implement CRF algorithm from scratch but rather use existing tools or libraries. There are several popular CRF toolkits:

- http://www.chokkan.org/software/crfsuite/: A fast implementation of Conditional Random Fields (CRFs)
- https://taku910.github.io/crfpp/ CRF++: Yet Another CRF toolkit

Several Python libraries provide support to CRFsuite, including ***python-crfsuite*** and ***sklearn-crfsuite***. We try both and they have similiar performance, but we prefer the ***sklearn-crfsuite*** since it has similar sklearn usage format.

CRF++ is a command line interface with features needed to be set in files and training data and output wrote to files. That was used by the Niptic Khmer Segmentations paper.

Other CRF implementations include:

- https://wapiti.limsi.fr/ : Wapiti - A simple and fast discriminative sequence labelling toolkit
- http://mallet.cs.umass.edu/ : MALLET includes tools for sequence tagging
- http://mallet.cs.umass.edu/grmm/general_crfs.php : General CRFs in GRMM
- http://flexcrfs.sourceforge.net/ : FlexCRFs -- Flexible Conditional Random Fields

The tool we use to segment our training data is from Niptict Khmer Segmentation tool using CRF: https://niptict.edu.kh/khmer-word-segmentation-tool/.

## Training Dataset

To get training data, we need a fairly large corpus for training. This data need to consist of segmented words that can be used for training.

We have crawled data on publicly avaible Khmer websites mainly news related.These data contain different categories like Cambodian related news, international, technologies and others.

Thanks to [niptict](https://niptict.edu.kh) which provide a segmentation tool with very good accuracy, we are able to use it to format our training data. Instead of hand segment these data or use the existing segmentation in the data with non-zero space which are not reliable, we just pass the text to this tool to segment and save the output. This saved us a lot of time and efforts.

Then we look at the output can hand corrected a subset of the articles. The correction is saved into our code as string substitution so we can apply them to all of the articles. This ensures that our corrections is consistent with all of the articles. The new result will be used as our training data set.

The data can be found in Github here:
https://github.com/phylypo/segmentation-crf-khmer/tree/master/data.

We have combined the downloadable files into different batch size (100, 500, 1000, 5000, and 10000 articles). Each batch is compressed to save space. This notebook will show how to download and decompressed the files for the training.

### File name format
The file name format starts with a unique 6 digit number as id, then follow by '_seg' or '_orig' to indicate original text or segmented text. Some new set of files has '_200b' in the file name to indicate that we are using \u200b unicode character as a separator rather than space. This new approach distinguish existing space that we don't need to predict useful for non-Khmer text.

Here are some examples:
- 451807_seg_200b.txt : segmented text using 200b character
- 451807_orig.txt : original text
- 123456_seg.txt : old format segmented text using space (will not use this).

## Approaches to Khmer Segmentation with CRF
The goal is to train Khmer word segmentation using Conditional Random Fields (CRF) which was implemented in the paper [1] showing 0.99 F-score. We compare the different implimentation approaches:
1. character based vs. Khmer Characters Cluster (KCC)
1. different ngram length features
1. CRF parameters tuning

We will detail the our approach to generating the training data, clean up the train data, generate the needed featrues for the algorithm, run and fine tune the algorithm and then measure the result.

#### May need to delete the following

The general idea is to predict if a given letter is the start of a word from a given Khmer text.

Fo example if we were given: <pre>This is a test.</pre>

First we remove the space and label the next character with 1 and the rest with zero. Then label as:
<pre>
Thisisatest.
100010110001
</pre>
where value 1 is for the char that will start as a word. The goal for the algorithm is to correctly guest where the value 1 will be given a string of characters in a sentence.

## Features Engineering

In this supervised machine learning algorithm, we need to create features so the algorithm can learn from. Fortunately, Linear chain CRF can take all kind of features without having to worry about feature correlation as in Naive Bayes or scaling.

So in a way, creating feature is fairly simple. But we need to make sure those features contribute to the performance of the algorithm.

We will cover several approaches:

* different ngram size
* different features (char type, char count)
* type of chars/kcc

Note: Python CRFsuite supports for features encoded in list or dictionary. It didn't make any differences but shorten the key, save lots of memory form "char" to "c".

First let's look at the features used by Vichet Chea et. al. paper. For each character, the paper identifies 10 different type of characters. These tags are consonant, vowel, independence vowel, upper sign, Atak number, lunar number, subscript, sentence sign, no space (numbers), and unknown.


Here are the tags and list of characters:

![tags used in Vichet Chea et. al. paper](https://cdn-images-1.medium.com/max/800/1*zzeP6cAgcsFc8vKDaIU9iQ.png)

The label that tags the character 
Tags:

![label tags in Vichet Chea et. al. paper](https://cdn-images-1.medium.com/max/800/1*k6cMZm2cbPv_ll62fsDQvA.png)

*** may need to remove this section ***
What is a word?
This in itself is a difficult thing to do. As native speaker, I have difficulty discerning a series of text as a compound word or not. 
Let's look at Chuon Nath dictionary first: has 17,664 head words
 18,947 entries with 17,664 head words. (each word may have multiple entries/definitions)
Keyword:


Compound Word
Prefix:
កតញ្ញូ vs អកតញ្ញូ (both words appear in dict) 
សំពះ (កិ.) ការ​សំពះ (ន.) (later not in dict) --without 200b ការសំពះ
ការ = សំពះ (កិ.) ការ​សំពះ (ន.) ជាដើម ។ 

Suffix:

We consider this as one word. For our context, we going to consider compound word as one word.  
Name: អកយំ ឈ្មោះ​បទ​ភ្លេង​មួយ​ប្រភេទ sad melody used in classical Cambodian music
អកអំបុក name of a ceremony
Cambodian-English Dictionary by Robert K. Headley 23,967 head word 33,543 sub-entries (word/phrase that contains the head word)
Cambodian Red Cross: កាកបាទក្រហមកម្ពុជា
( https://www.redcross.org.kh)
CRF can address long series:
ខុំញ ចង់ឱយ < អនក􀇒􀆎 ប់ > យល់ ពី ប􀈦􀆟 េនះ
I want the listener to understand this problem

"listener" vs "you to listen":

Here is our example features with character approach:

<pre>តើអ្នកឈ្មោះអ្វី?</pre> (translated as "What is your name?")
 

| char | type | label | segmented word |
|---------|---------|:--------:| :---: | 
| ត   | C   | 1 | តើ
| ⁣ើ   | V  | 0 |
| អ   | C   | 1 | ⁣អ្នក
| ◌្   | S   | 0| 
| ន   | C   | 0 |
| ក   | C   | 0 |
| ឈ | C   | 1 | ⁣ឈ្មោះ
| ◌្   | S | 0 |
| ម   | C   | 0 |
| ោ | V  |  0 |
|◌ះ   | V   | 0 |
| អ   | V   | 1 | ⁣អ្វី
|  ្  | S  | 0 |
|  វ  | C   | 0 |
|  ី  | V   | 0 |
|  ?  | O   | 1 | ⁣?





Note on some differences to the implimentation in CRF paper:
1. label at ending instead of beginning -- just preference
2. type combine -- on consonant (didn't see the value of the differences)
3. not doing any tags (2 tags, 5 tags)


Notes on some of the differences to the CRF paper:
* instead of end of word, we use beginning of the word
* type is slighly differences
* Use ngram characters (not sure if this was implemented in the paper)

For KCC:
- type would be: K=Khmer text (contain consonant/vowels), N:number, E=Non-khmer, U:unknown

### KCC Rules

This is Khmer Character Cluster (KCC) rule in BNF format:

>  <C|I> + [<Robat | Regshift>] + {COEUNG + <C + [Regshift] | I + [Regshift]>} + [[<ZWJ|ZWNJ>] +
V] + {S} + [ZWJ + COEUNG + <C | I>]


| Symbols | Meaning |
| -- | --- |
| {} | Zero to two occurrences
| [ ] | Zero to one occurrences
| <x \| y> | The choice of x or y |
| C | Consonant
| I |  Independent vowel
| COEUNG | The COEUNG character (\u 17D2)
| V | Vowel
| Regshift |  Registry shifter
| S | Various sign
| ZWNJ |  ZERO WIDTH NON-JOINERS
| ZWJ | ZERO WIDTH JOINERS
| ROBAT | The Robat sign (\u 17CC) 

Source: [Detection and Correction of Homophonous Error Word for Khmer Language](http://ww.panl10n.net/english/outputs/Working%20Papers/Cambodia/Microsoft%20Word%20-%204_E_N_248.pdf)
    
Here is an example: 

Text: សាស្ត្រា​ត្រីនេត្រ 

Word: សាស្ត្រា - ត្រីនេត្រ 

KCC: សា - ស្ត្រា - ត្រី - នេ - ត្រ 


Our implementation of KCC is a lot simpler than the rule. We just find the beginning of the character which is either C or I that does not follow by COEUNG.

## Results and Analysis



Comparison:
Vichet Chear paper has more features:
- prefix, suffix, compound word notation
This means the algorithm has predicts more set of classes (more possible outcomes).

we don't have any extra notation so our labels is just 2 classes 0 or 1 which is enough for our purpose.


More domain data such as: religious domain, economic, and medical. Our training and test set are all from one news domain even though it cover variety of topics.





### KCC vs Character based
Here we try to compare the result using KCC and Character based.
The accuracy result on test set for KCC is lower than character based:
- 97.37% for KCC
- 97.5% for Character

This is because KCC has lower number of tokens so ratio of correct/incorrect is lower. By looking at word accuracy, we see that KCC does perform better:
- 96.87% for KCC
- 95.37% for character

Word accuracy is a better comparison since the ratio is of the same scale. Thus we choose KCC for the full dataset training.

The KCC approach able to learn very fast where the training set result if almost perfect with 0.9997 and character approach is 0.987. KCC has more possible state (combination of different characters --1836  kcc) where character approach is only 176 different characters.  


#### Detail result for KCC vs Character based

KCC based
```
=== Training set performance === 
total kcc: 57964  total word: 24704
correct kcc:57950  incorrect kcc: 14  kcc accuracy: 0.999758
correct word:24695  missed word: 9 word accuracy: 0.999635
Precision:	0.999797
Recall:		0.999635
Accuracy:	0.999758

              precision    recall  f1-score   support

           0       1.00      1.00      1.00     33260
           1       1.00      1.00      1.00     24704

    accuracy                           1.00     57964
   macro avg       1.00      1.00      1.00     57964
weighted avg       1.00      1.00      1.00     57964


=== Test set performance === 
total kcc: 15271  total word: 6512
correct kcc: 14870  incorrect kcc: 401  kcc accuracy: 0.973741
correct word: 6308  missed word: 204 word accuracy: 0.968673
Precision: 0.969715
Recall:    0.968673
Accuracy:  0.973741

              precision    recall  f1-score   support

           0       0.98      0.98      0.98      8759
           1       0.97      0.97      0.97      6512

    accuracy                           0.97     15271
   macro avg       0.97      0.97      0.97     15271
weighted avg       0.97      0.97      0.97     15271

```

Character Based

```
Training set --------------- 
size of input: 800
Total char: 125974  total word: 26727
correct char:124416  incorrect: 1558 char accuracy: 0.9876323685839935
correct word:26022  missed word: 705 word accuracy: 0.9736221798181615
n_edit: 1553  acc: 0.9876720593138266
Precision:	0.9682604651162791
Recall:		0.9736221798181615
Accuracy:	0.9876323685839935

Test set --------------- 
size of input: 108
Total char: 16946  total word: 3605
correct char:16526  incorrect: 420 char accuracy: 0.9752153900625516
correct word:3438  missed word: 167 word accuracy: 0.9536754507628294
n_edit: 419  acc: 0.9752744010385932
Precision:	0.9314548902736386
Recall:		0.9536754507628294
Accuracy:	0.9752153900625516
```


### Comparing the Final Result

Here we will analyze the result in comparison to the latest state of the art from Vichet Chea et al. Our implementation here has an error rate of 0.3% while the performance from Vichet Chear et al. [2] is 98.5% accuracy or 1.5% error rate. 

Our implementation is significantly better in term of performance but our implementation only considers 2 tags (1: a beginning of a word, 0: otherwise) while Vichet Chea et al. [2] has 5 different tags to distinguish the prefix, suffix and compound word. So there are more options for the algorithm in [2] to learn and predict thus it is much harder.

Although, [2] shows the experiment result on 2 tags vs 5 tags has a very similar F-score implying that performance are not too different.

Also note that the accuracy for paper is based on number of correct characters over total number of characters while our implementation is based on KCC.


## Conclusion

We compared KCC vs character based implementation and choose KCC for better word accuracy.

We cover steps to fine tune the hyper parameter. We did a few options for different features but we did not show all the steps. But shorten the key is one main approach to save memory for training.

We are able to achieve a 0.3% error rate.

## References

### Source


Data:
- bible word breaking: https://github.com/silnrsi/khmerlbdict --has sealang freq word: http://sealang.net/project/list/
- using tries: https://viblo.asia/p/nlp-khmer-word-segmentation-YWOZrgNNlQ0
- https://github.com/RathanakSreang/KhmerWordSegmentation

CFR Paper
* https://www.research.ed.ac.uk/portal/files/10482724/crftut_fnt.pdf
An Introduction to Conditional Random Fields, Sutton, C & McCallum
* https://github.com/wukuun/crf/blob/master/crfpaper/crf_klinger_tomanek.pdf
* http://www.albertauyeung.com/post/python-sequence-labelling-with-crf/
* https://medium.com/@felixmohr/using-python-and-conditional-random-fields-for-latin-word-segmentation-416ca7a9e513
  * https://github.com/FelixMohr/NLP-with-Python
  * word seg: https://github.com/FelixMohr/NLP-with-Python/blob/master/CRFs-latin-word-segmenation.ipynb
* simple crf implementation:  https://github.com/timvieira/crf/tree/master/crf
* http://alias-i.com/lingpipe/demos/tutorial/crf/read-me.html

Khmer References
* Vichet Chea, Ye Kyaw Thu, Chenchen Ding, Masao Utiyama, Andrew Finch, Eiichiro Sumita. Khmer Word Segmentation Using Conditional Random Fields. Research and Development Center, NIPTICT, Phnom Penh, Cambodia. Dec 2015. https://pdfs.semanticscholar.org/818e/213b403d6c8382fab6dd67d2679ef198d334.pdf
* Chea Sok Huor, Top Rithy, Ros Pich Hemy, Vann Navy, “Word Bigram Vs Orthographic Syllable Bigram in Khmer Word Segmentation” http://ww.panl10n.net/english/final%20reports/pdf%20files/Cambodia/CAM05.pdf
* A Large-scale Study of Statistical Machine Translation Methods
for Khmer Language: https://www.aclweb.org/anthology/Y15-1030 (Ye Kyaw Thu, Vichet Chea)

